In [ ]:
import pickle 

import imp
from IPython.display import clear_output, display
import matplotlib
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import os

from context import rf_pool

In [ ]:
from rf_pool import models, modules, pool, ops
from rf_pool.utils import lattice, functions, visualize, datasets, stimuli

In [ ]:
from experiment_functions import * 

In [ ]:
directories = ['results', 'datasets']
for d in directories:
    if not os.path.exists(d):
        os.mkdir(d)

**Load MNIST Data**

In [ ]:
# get MNIST training data
transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.MNIST(root='../../data', train=True, download=True, 
                                       transform=transform)
testset = torchvision.datasets.MNIST(root='../../data', train=False, download=True,
                                     transform=transform)

In [ ]:
# create trainloader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=1,
                                         shuffle=True, num_workers=2)

In [ ]:
# load in crowded digits base set
base_set_filename = 'MNIST_CrowdedDataset.pkl'
if os.path.exists('datasets/' + base_set_filename):
    base_set = pickle.load(open('datasets/' + base_set_filename, 'rb'))
else:
    base_set = None
    print('Base Set not found!')
    
# set what labels mmap to what digit
label_map = {}
label_map.update([(n,n) for n in range(10)])

**Load Model**

In [ ]:
# initialize model
model = rf_pool.models.FeedForwardNetwork()

In [ ]:
# append layers of model
model.append('0', rf_pool.modules.FeedForward(hidden=torch.nn.Conv2d(1,32,5),
                                              activation=torch.nn.ReLU(), 
                                              pool=torch.nn.MaxPool2d(2)))
model.append('1', rf_pool.modules.FeedForward(hidden=torch.nn.Conv2d(32,64,5),
                                              activation=torch.nn.ReLU(),
                                              pool=torch.nn.MaxPool2d(2)))
model.append('2', rf_pool.modules.FeedForward(hidden=torch.nn.Conv2d(64,10,4)))
model.append('3', rf_pool.modules.FeedForward(input_shape=(-1,10)))

In [ ]:
# load previous model and results
(_, extras) = model.load_model('models/MNIST_rate_0.2_10k_3deg.pkl')

In [ ]:
# remove reshape layer 
model.layers.pop('3')

**Replace max pool layer with rf pool layer**

In [ ]:
# create the rf layer
img_shape = torch.Size((53,53))
offset = [0., -30.] # right visual field (3deg)
ref_axis = 0. # set the reference axis for the crowding configurations
RF_rate = 0.2
gap = 0.
n_rings = 10
std = 1.
mu, sigma = rf_pool.utils.lattice.init_foveated_lattice(img_shape, RF_rate, gap, n_rings=n_rings, std=std,
                                                        offset=offset)
rf_layer = rf_pool.pool.RF_Pool(mu=mu, sigma=sigma, img_shape=img_shape, 
                                lattice_fn=rf_pool.utils.lattice.mask_kernel_lattice,
                                pool_fn='max_pool', kernel_size=2)
n_kernels = rf_layer.mu.shape[0]
# append the rf pool layer to the model
layer_id = '1'
model.layers[layer_id].forward_layer.add_module('pool', rf_layer)
visualize.heatmap(model, '1');

Get redundancy as proportion overlap within target square

**Test Attention**

In [ ]:
# set experiment parameters
exp_type = 'Attention'
n_test = 1000
batch_size = 1

# set lattice kwargs
img_shape = torch.Size((53,53))
RF_rate = 0.2
gap = 0.
n_rings = 10
std = 1.

# get angle between RFs
n_RF = np.floor(np.pi / RF_rate)
angles = 2. * np.pi * np.linspace(0., 1., np.int(n_RF))[:-1]
rot_angle = angles[1]

In [ ]:
extents = np.arange(7.,29.5,2.5)
spacing = 1.
crowd_types = ['outer','inner','radial','tangential']
# run the experiment
RF_ACC = {}
for key in crowd_types:
    n_flankers, axis = get_crowd_params(key, ref_axis)
    ACC = []
    for extent in extents:
        clear_output(wait=True)
        display('%s: %a' % (key, extent))
        # get target data
        target_set = create_crowd_set(testset, n_test, 118, 0, 0, 0, base_set=base_set, label_map=label_map,
                                      transform=transforms.Compose([transforms.ToTensor(), jitter_fn(5, 5, seed=0)]))
        target_loader = torch.utils.data.DataLoader(target_set, batch_size=batch_size,
                                                    shuffle=False, num_workers=2)
        # get crowd data
        crowd_set = create_crowd_set(testset, n_test, 118, n_flankers, axis, spacing, base_set, label_map,
                                     transform=transforms.Compose([transforms.ToTensor(), jitter_fn(5, 5, seed=0)]))
        crowd_loader = torch.utils.data.DataLoader(crowd_set, batch_size=batch_size,
                                                   shuffle=False, num_workers=2)
        lattice_fn = lattice.init_foveated_lattice
        lattice_kwargs = {'img_shape': img_shape, 'scale': RF_rate, 'spacing': gap, 
                          'std': std, 'n_rings': n_rings, 'offset': offset, 
                          'rotate': rotate_fn(rot_angle / 2., base_angle=ref_axis, seed=0)}
        # Get accuracy
        ACC.append(get_accuracy(target_loader, crowd_loader,
                                **{'layer_id':'1', 'batch_size': batch_size, 
                                   'model': model, 'extent': extent, 
                                   'lattice_fn': lattice_fn,
                                   'lattice_kwargs': lattice_kwargs}))
    RF_ACC.update({key + '_attn': ACC, 'extent': extents})

In [ ]:
with open('results/ACC_%s_%dk.pkl' % (exp_type.lower(), n_test/1000), 'wb') as f:
    pickle.dump(RF_ACC, f)

**Test Spacing**

In [ ]:
# set experiment parameters
exp_type = 'Spacing'
n_test = 1000
batch_size = 1

# set lattice kwargs
img_shape = torch.Size((53,53))
RF_rate = 0.2
gap = 0.
n_rings = 10
std = 1.

# get angle between RFs
n_RF = np.floor(np.pi / RF_rate)
angles = 2. * np.pi * np.linspace(0., 1., np.int(n_RF))[:-1]
rot_angle = angles[1]

In [ ]:
spacings = np.arange(1., 2.25, 0.25)
crowd_types = ['outer','inner','radial','tangential']
# run the experiment
RF_ACC = {}
for key in crowd_types:
    n_flankers, axis = get_crowd_params(key, ref_axis)
    ACC = []
    for spacing in spacings:
        clear_output(wait=True)
        display('%s: %a' % (key, spacing))
        # get target data
        target_set = create_crowd_set(testset, n_test, 118, 0, 0, 0, base_set=base_set, label_map=label_map,
                                      transform=transforms.Compose([transforms.ToTensor(), jitter_fn(5, 5, seed=0)]))
        target_loader = torch.utils.data.DataLoader(target_set, batch_size=batch_size,
                                                    shuffle=False, num_workers=2)
        # get crowd data
        crowd_set = create_crowd_set(testset, n_test, 118, n_flankers, axis, spacing, base_set, label_map,
                                     transform=transforms.Compose([transforms.ToTensor(), jitter_fn(5, 5, seed=0)]))
        crowd_loader = torch.utils.data.DataLoader(crowd_set, batch_size=batch_size,
                                                   shuffle=False, num_workers=2)
        lattice_fn = lattice.init_foveated_lattice
        lattice_kwargs = {'img_shape': img_shape, 'scale': RF_rate, 'spacing': gap, 
                          'std': std, 'n_rings': n_rings, 'offset': offset, 
                          'rotate': rotate_fn(rot_angle / 2., base_angle=ref_axis, seed=0)}
        # Get accuracy
        ACC.append(get_accuracy(target_loader, crowd_loader,
                                **{'layer_id':'1', 'batch_size': batch_size, 
                                   'model': model, 'extent': None, 
                                   'lattice_fn': lattice_fn,
                                   'lattice_kwargs': lattice_kwargs}))
    RF_ACC.update({key + '_space': ACC, 'spacing': spacings})

In [ ]:
with open('results/ACC_%s_%dk.pkl' % (exp_type.lower(), n_test/1000), 'wb') as f:
    pickle.dump(RF_ACC, f)

**Test Size/Position Attention**

In [ ]:
# set experiment parameters
exp_type = 'Attention_Size_Position'
n_test = 1000
batch_size = 1

# set lattice kwargs
img_shape = torch.Size((53,53))
RF_rate = 0.2
gap = 0.
n_rings = 10
std = 1.

# get angle between RFs
n_RF = np.floor(np.pi / RF_rate)
angles = 2. * np.pi * np.linspace(0., 1., np.int(n_RF))[:-1]
rot_angle = angles[1]

In [ ]:
extents = np.arange(7.,29.5,2.5)
spacing = 1.
crowd_types = ['radial','tangential', 'outer', 'inner']
# run the experiment
RF_ACC = {}
for key in crowd_types: 
    ACC = []
    # set update_mu, update_sigma to True/False
    for update_mu in [True, False]:
        update_sigma = (not update_mu)
        # get target data
        target_set = create_crowd_set(testset, n_test, 118, 0, 0, 0, base_set=base_set, label_map=label_map)
        target_loader = torch.utils.data.DataLoader(target_set, batch_size=batch_size,
                                                    shuffle=False, num_workers=2)
        # get crowd data
        n_flankers, axis = get_crowd_params(key, ref_axis)
        crowd_set = create_crowd_set(testset, n_test, 118, n_flankers, axis, spacing,
                                     base_set, label_map)
        crowd_loader = torch.utils.data.DataLoader(crowd_set, batch_size=batch_size,
                                                       shuffle=False, num_workers=2)
        param_space = extents
        param_name = ['extent']
        lattice_fn = lattice.init_foveated_lattice
        lattice_kwargs = {'img_shape': img_shape, 'scale': RF_rate, 'spacing': gap, 
                          'std': std, 'n_rings': n_rings, 'offset': offset,
                          'rotate': rotate_fn(rot_angle / 2., base_angle=ref_axis, seed=0)}
        # Get accuracy
        ACC.append(get_accuracy(target_loader, crowd_loader, 
                                **{'model': model, 'batch_size': 1, 
                                   'layer_id':'1', 'extent': extent,
                                   'lattice_fn': lattice_fn, 
                                   'lattice_kwargs': lattice_kwargs,
                                   'update_mu': update_mu, 'update_sigma': update_sigma}))
        # monitor progress
        clear_output(wait=True)
        display('%s: mu %a, sigma %a' % (key, update_mu, update_sigma))
    RF_ACC.update({key + '_attn': ACC, 'extent': extents})

In [ ]:
with open('results/ACC_%s_%dk.pkl' % (exp_type.lower(), n_test/1000), 'wb') as f:
    pickle.dump(RF_ACC, f)

**Get Heatmaps**

In [ ]:
import experiment_functions
imp.reload(experiment_functions)
from experiment_functions import *

In [ ]:
# set experiment parameters
exp_type = 'Heatmaps'
n_test = 1000
batch_size = 1
# set lattice kwargs
img_shape = torch.Size((53,53))
RF_rate = 0.2
gap = 0.
n_rings = 10
std = 1.

# get angle between RFs
n_RF = np.floor(np.pi / RF_rate)
angles = 2. * np.pi * np.linspace(0., 1., np.int(n_RF))[:-1]
rot_angle = angles[1]

In [ ]:
spacings = np.arange(1., 2.25, 0.25)
extents = [None] + np.arange(7.,29.5,2.5).tolist()
crowd_types = ['inner', 'outer', 'radial', 'tangential']
# run the experiment
RF_heatmaps = {}
# get accuracy heatmaps
for key in crowd_types:
    n_flankers, axis = get_crowd_params(key, ref_axis)
    heatmap = []
    for extent in extents:
        for spacing in spacings:
            # if extent and spacing > 1, skip
            if extent is not None and spacing > 1.:
                continue
            # get target data
            target_set = create_crowd_set(testset, n_test, 118, 0, 0, 0, base_set=base_set, label_map=label_map,
                                          transform=transforms.Compose([transforms.ToTensor(), jitter_fn(5, 5, seed=0)]))
            target_loader = torch.utils.data.DataLoader(target_set, batch_size=batch_size,
                                                        shuffle=False, num_workers=2)
            # get crowding set (without target to account for flankers in get_contribution call)
            crowd_set = create_crowd_set(testset, n_test, 118, n_flankers, axis, spacing, base_set, label_map,
                                         no_target=True,
                                         transform=transforms.Compose([transforms.ToTensor(),
                                                                       jitter_fn(5, 5, seed=0)]))
            crowd_loader = torch.utils.data.DataLoader(crowd_set, batch_size=batch_size,
                                                       shuffle=False, num_workers=2)
            # set lattice kwargs 
            lattice_fn = lattice.init_foveated_lattice
            lattice_kwargs = {'img_shape': img_shape, 'scale': RF_rate, 'spacing': gap, 
                              'std': std, 'n_rings': n_rings, 'offset': offset, 
                              'rotate': 0.}
            # get rf scores
            heatmap.append(get_confidence(target_loader, crowd_loader,
                                            **{'layer_id': layer_id,
                                               'model': model,
                                               'RF_mask': None, 
                                               'acc': None,
                                               'extent': extent,
                                               'lattice_fn': lattice_fn,
                                               'lattice_kwargs': lattice_kwargs}))
            # monitor progress
            clear_output(wait=True)
            display('%s: spacing %a, extent %a' % (key, spacing, extent))
    # append heatmaps
    RF_heatmaps.update({key + '_heatmap': heatmap, 'spacing': spacings, 'extent': extents})

In [ ]:
with open('results/confidence_%s_%dk.pkl' % (exp_type.lower(), n_test/1000), 'wb') as f:
    pickle.dump(RF_heatmaps, f)

In [ ]:
RF_heatmaps = pickle.load(open('results/confidence_heatmaps_1k.pkl', 'rb'))

In [ ]:
key = 'tangential'
exp_type = 'attention'
s = len(RF_heatmaps['spacing']) * (exp_type == 'attention') + (exp_type == 'spacing')
e = len(RF_heatmaps['spacing']) + (len(RF_heatmaps['extent']) - 1) * (exp_type == 'attention')
m = len(RF_heatmaps['spacing']) - (exp_type == 'spacing')
avg_heatmap = torch.mean(torch.stack(RF_heatmaps['%s_heatmap' % key][s:e]), 0)
# max_heatmap = RF_heatmaps['%s_heatmap' % key][m]
scores = avg_heatmap - RF_heatmaps['%s_heatmap' % key][0]
for i, v in enumerate(scores):
    if v == 0.:
        scores[i] = np.nan
fig = visualize.heatmap(model, layer_id, scores=scores, cmap='RdYlGn', colorbar=True, vmin=-0.1, vmax=0.1,
                        figsize=(4.9,4.9))
fig.suptitle('%s Avg - %s %s' % (key, ['Min', 'No'][exp_type=='attention'], exp_type))
fig

In [ ]:
confidence_sums = {}
keys = ['outer','inner','radial','tangential']
colors = ['blue', 'orange','green','red']
for i, key in enumerate(keys):
    sums = []
    confidence_sums.update({key: sums})
    for ii in range(len(RF_heatmaps['spacing']), len(RF_heatmaps['%s_heatmap' % key])):
        sums.append(np.sum([v for v in RF_heatmaps['%s_heatmap' % key][ii] if not torch.isnan(v)]))
    plt.plot(RF_heatmaps['extent'][1:], sums, color=colors[i])
plt.legend(keys)

**Get Average Area of FWHM Importance Map**

In [ ]:
import experiment_functions
imp.reload(experiment_functions)
from experiment_functions import *

In [ ]:
# set experiment parameters
exp_type = 'spread'
n_test = 10000
batch_size = 1
# set lattice kwargs
img_shape = torch.Size((53,53))
RF_rate = 0.2
gap = 0.
n_rings = 10
std = 1.

# get angle between RFs
n_RF = np.floor(np.pi / RF_rate)
angles = 2. * np.pi * np.linspace(0., 1., np.int(n_RF))[:-1]
rot_angle = angles[1]

In [ ]:
spacings = np.arange(1., 2.25, 0.25)
extents = [None] + np.arange(7.,29.5,2.5).tolist()
crowd_types = ['inner', 'outer', 'radial', 'tangential']
# run the experiment
importance_spread = {}
# get accuracy heatmaps
for key in crowd_types:
    n_flankers, axis = get_crowd_params(key, ref_axis)
    spreads = []
    for extent in extents:
        for spacing in spacings:
            # if extent and spacing > 1, skip
            if extent is not None and spacing > 1.:
                continue
            # get target data
            target_set = create_crowd_set(testset, n_test, 118, 0, 0, 0, base_set=base_set, label_map=label_map,
                                          transform=transforms.Compose([transforms.ToTensor(), jitter_fn(5, 5, seed=0)]))
            target_loader = torch.utils.data.DataLoader(target_set, batch_size=batch_size,
                                                        shuffle=False, num_workers=2)
            # get flank set (without target to account for flankers in get_contribution call)
            flank_set = create_crowd_set(testset, n_test, 118, n_flankers, axis, spacing, base_set, label_map,
                                         no_target=True,
                                         transform=transforms.Compose([transforms.ToTensor(),
                                                                       jitter_fn(5, 5, seed=0)]))
            flank_loader = torch.utils.data.DataLoader(flank_set, batch_size=batch_size,
                                                       shuffle=False, num_workers=2)
            # set lattice kwargs 
            lattice_fn = lattice.init_foveated_lattice
            lattice_kwargs = {'img_shape': img_shape, 'scale': RF_rate, 'spacing': gap, 
                              'std': std, 'n_rings': n_rings, 'offset': offset, 
                              'rotate': rotate_fn(rot_angle / 2., base_angle=ref_axis, seed=0)}
            # get importance maps
            maps_i = get_importance_map(target_loader, flank_loader,
                                        **{'layer_id': layer_id,
                                           'model': model,
                                           'extent': extent,
                                           'lattice_fn': lattice_fn,
                                           'lattice_kwargs': lattice_kwargs})
            # get average area FWHM
            maps_i = torch.squeeze(torch.stack(maps_i))
            fwhm = torch.max(torch.flatten(maps_i, -2), -1)[0] / 2.
            spreads.append(torch.mean(torch.sum(torch.ge(maps_i, fwhm.reshape(-1, 1, 1)), [1,2]).float()))
            # monitor progress
            clear_output(wait=True)
            display('%s: spacing %a, extent %a' % (key, spacing, extent))
    # append heatmaps
    importance_spread.update({key + '_spread': spreads, 'spacing': spacings, 'extent': extents})

In [ ]:
with open('results/importance_%s_%dk.pkl' % (exp_type.lower(), n_test/1000), 'wb') as f:
    pickle.dump(RF_heatmaps, f)

**Get Redundancy**

In [ ]:
import experiment_functions
imp.reload(experiment_functions)
from experiment_functions import *

In [ ]:
# set experiment parameters
exp_type = 'Redundancy'
n_test = 1000
batch_size = 1
# set lattice kwargs
img_shape = torch.Size((53,53))
RF_rate = 0.2
gap = 0.
n_rings = 10
std = 1.

# get angle between RFs
n_RF = np.floor(np.pi / RF_rate)
angles = 2. * np.pi * np.linspace(0., 1., np.int(n_RF))[:-1]
rot_angle = angles[1]

In [ ]:
spacings = [1.]
extents = np.arange(7.,29.5,2.5).tolist()
crowd_types = ['']
# run the experiment
RF_redundancy = {}
# get accuracy heatmaps
for key in crowd_types:
    n_flankers, axis = get_crowd_params(key, ref_axis)
    redundancy = []
    for extent in extents:
        for spacing in spacings:
            # if extent and spacing > 1, skip
            if extent is not None and spacing > 1.:
                continue
            # get target data
            target_set = create_crowd_set(testset, n_test, 118, 0, 0, 0, base_set=base_set, label_map=label_map,
                                          transform=transforms.Compose([transforms.ToTensor(), jitter_fn(5, 5, seed=0)]))
            target_loader = torch.utils.data.DataLoader(target_set, batch_size=batch_size,
                                                        shuffle=False, num_workers=2)
            # create square data set to get redundancy metric
            s = stimuli.make_crowded_stimuli(torch.ones(20,20), [], 0., (118,118))
            square_set = datasets.Dataset(data=[s], transform=transforms.Compose([transforms.ToTensor(), 
                                          jitter_fn(5, 5, seed=0)]))
            square_loader = torch.utils.data.DataLoader(square_set, batch_size=1, shuffle=False, num_workers=2)
            # get crowding set (without target to account for flankers in get_contribution call)
            crowd_set = create_crowd_set(testset, n_test, 118, n_flankers, axis, spacing, base_set, label_map,
                                         no_target=True,
                                         transform=transforms.Compose([transforms.ToTensor(),
                                                                       jitter_fn(5, 5, seed=0)]))
            crowd_loader = torch.utils.data.DataLoader(crowd_set, batch_size=batch_size,
                                                       shuffle=False, num_workers=2)
            # set lattice kwargs 
            lattice_fn = lattice.init_foveated_lattice
            lattice_kwargs = {'img_shape': img_shape, 'scale': RF_rate, 'spacing': gap, 
                              'std': std, 'n_rings': n_rings, 'offset': offset, 
                              'rotate': rotate_fn(rot_angle / 2., base_angle=0, seed=0)}
            # get redundancy
            redundancy.append(get_redundancy(target_loader, crowd_loader, square_loader,
                                            **{'layer_id': layer_id,
                                               'model': model,
                                               'RF_mask': None, 
                                               'acc': None,
                                               'extent': extent,
                                               'lattice_fn': lattice_fn,
                                               'lattice_kwargs': lattice_kwargs}))
            # monitor progress
            clear_output(wait=True)
            display('%s: spacing %a, extent %a' % (key, spacing, extent))
    # append heatmaps
    RF_redundancy.update({key + 'redundancy': redundancy, 'spacing': spacings, 'extent': extents})

In [ ]:
with open('results/RF_redundancy_10k.pkl', 'wb') as f:
    pickle.dump(RF_redundancy, f)

**Get Correlation of Accuracy and Redundancy**

In [ ]:
import scipy.stats as stats

In [ ]:
acc = pickle.load(open('results/ACC_attention_size_position_10k.pkl', 'rb'))

In [ ]:
RF_redundancy = pickle.load(open('results/RF_redundancy_10k.pkl', 'rb'))

In [ ]:
key = 'inner'
stats.pearsonr([v[0] for v in acc['%s_attn' % key][1]], RF_redundancy['redundancy'])

In [ ]:
key = 'tangential'
stats.pearsonr(confidence_sums['%s' % key], RF_redundancy['redundancy'])

In [ ]:
accs = []
reds = []
for key in ['inner', 'outer', 'radial', 'tangential']:
    accs.extend([v[0] for v in acc['%s_attn' % key][1]])
    reds.extend(RF_redundancy['redundancy'])
stats.pearsonr(accs, reds)

**Get accuracy for all conditions**

In [ ]:
# set experiment parameters
exp_type = 'Accuracy'
n_test = 1000
batch_size = 1

# set lattice kwargs
img_shape = torch.Size((53,53))
RF_rate = 0.2
gap = 0.
n_rings = 10
std = 1.

# get angle between RFs
n_RF = np.floor(np.pi / RF_rate)
angles = 2. * np.pi * np.linspace(0., 1., np.int(n_RF))[:-1]
rot_angle = angles[1]

In [ ]:
spacings = np.linspace(1., 2., 9).tolist()
extents = np.arange(7.,29.5,2.5).tolist()
crowd_types = ['inner', 'outer', 'radial', 'tangential']
# run the experiment
RF_ACC = {}
# get accuracy heatmaps
for key in crowd_types:
    n_flankers, axis = get_crowd_params(key, ref_axis)
    ACC = {}
    for extent in extents:
        for spacing in spacings:
            # get target data
            target_set = create_crowd_set(testset, n_test, 118, 0, 0, 0, base_set=base_set, label_map=label_map,
                                          transform=transforms.Compose([transforms.ToTensor(), jitter_fn(5, 5, seed=0)]))
            target_loader = torch.utils.data.DataLoader(target_set, batch_size=batch_size,
                                                        shuffle=False, num_workers=2)
            # get crowding set (without target to account for flankers in get_contribution call)
            crowd_set = create_crowd_set(testset, n_test, 118, n_flankers, axis, spacing, base_set, label_map,
                                         transform=transforms.Compose([transforms.ToTensor(),
                                                                       jitter_fn(5, 5, seed=0)]))
            crowd_loader = torch.utils.data.DataLoader(crowd_set, batch_size=batch_size,
                                                       shuffle=False, num_workers=2)
            # set lattice kwargs 
            lattice_fn = lattice.init_foveated_lattice
            lattice_kwargs = {'img_shape': img_shape, 'scale': RF_rate, 'spacing': gap, 
                              'std': std, 'n_rings': n_rings, 'offset': offset, 
                              'rotate': rotate_fn(rot_angle / 2., base_angle=ref_axis, seed=0)}
            # get rf scores
            ACC.update({'space_%d_attn_%d' % (spacing, extent): get_accuracy(target_loader, crowd_loader,
                                                                            **{'layer_id': layer_id,
                                                                               'model': model,
                                                                               'RF_mask': None,
                                                                               'extent': extent,
                                                                               'lattice_fn': lattice_fn,
                                                                               'lattice_kwargs': lattice_kwargs})})
            # monitor progress
            clear_output(wait=True)
            display('%s: spacing %a, extent %a' % (key, spacing, extent))
    # append heatmaps
    RF_ACC.update({key + '_ACC': ACC, 'spacing': spacings, 'extent': extents})

In [ ]:
with open('results/RF_%s_%dk.pkl' % (exp_type.lower(), n_test/1000), 'wb') as f:
    pickle.dump(RF_ACC, f)